<a href="https://colab.research.google.com/github/imabari/rakuten_tools/blob/main/enb_lcid_package.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TowerCollectorのCSVを選択

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [ ]:
import io
import pandas as pd

In [ ]:
pd.set_option("display.max_columns", None)

In [ ]:
df0 = (
    pd.read_csv(fn, parse_dates=["measured_at"])
    .query("mcc == 440 & mnc == 11 & ta < 5 & (188743680 <= cell_id < 192937984)")
    .dropna(how="all", axis=1)
    .sort_values(by=["cell_id", "rsrp"])
    .drop_duplicates(subset=["cell_id"])
    .reset_index(drop=True)
)
df0

In [ ]:
df0["eNB-LCID"] = df0["short_cell_id"].astype(str).str.cat(df0["rnc"].astype(str), sep="-")

In [ ]:
print(df0[["eNB-LCID", "psc"]].to_csv(index=False, header=False))

# 範囲をコピペしてから「以降のセルを実行」

In [ ]:
s = """
737451-1,80
737451-2,124
737451-3,3
"""

In [ ]:
df1 = pd.read_csv(io.StringIO(s), names=["eNB-LCID", "PCI"], dtype=str)
df1

In [ ]:
# eNBとLCIDに分割
df1[["eNB", "LCID"]] = df1["eNB-LCID"].str.split("-", expand=True)

# LCIDとPCIを結合
df2 = df1.groupby("eNB")[["LCID", "PCI"]].agg(lambda x: ",".join(x)).reset_index()

# eNBとLCIDを結合
df2["eNB-LCID"] = df2["eNB"].str.cat(df2["LCID"], sep="-")

In [ ]:
# eNB-LCIDとPCIのみ
df3 = df2.reindex(columns=["eNB-LCID", "PCI"])
df3

In [ ]:
# 表示
for i, r in df3.iterrows():
    print(r["eNB-LCID"])
    print(r["PCI"])
    print("-" * 20)